# Connect to MongoDB

In [11]:
import pymongo
# Connect to your MongoDB instance
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["vaers"]
collection = db["reports cleaned"]

# define global functions

In [12]:
from datetime import datetime

def convert_date(date_string):
    if pd.isna(date_string):
        return None
    # Assuming the date format is day/month/year
    return datetime.strptime(date_string, "%m/%d/%Y")

def calculate_age(row):
    if not pd.isna(row['AGE_YRS']):
        return row['AGE_YRS']
    elif not pd.isna(row['CAGE_YR']):
        age_years = row['CAGE_YR']
        if not pd.isna(row['CAGE_MO']):
            age_years += row['CAGE_MO']  # Convert months to years and add
        return age_years
    elif not pd.isna(row['CAGE_MO']):
        return row['CAGE_MO']   # Convert months to years
    else:
        return np.nan
    
def form_completed(row):
    if not pd.isna(row['TODAYS_DATE']):
        return row['TODAYS_DATE']
    elif not pd.isna(row['RPT_DATE']):
        return row['RPT_DATE']
    else:
        return np.nan
    
corrections = {
    'penecellin': 'Penicillin',
    'penecillin': 'Penicillin',
    'penecilin': 'Penicillin',
    'penicillins': 'Penicillin',
    'sulfa': 'Sulfates',
    'sulpha': 'Sulfates',
    'sulfides': 'Sulfates',
    'sulfite': 'Sulfates',
    'sulfate': 'Sulfates'
}

def correct_spelling(text):
    for variation, correction in corrections.items():
        text = re.sub(r'\b{}\b'.format(variation), correction, text, flags=re.IGNORECASE)
    return text

# Define the replacement function
def replace_vaccination_site(symptom):
    return symptom.str.replace("Vaccination site", "Injection site", flags=re.IGNORECASE)

# Domestic Reports

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm  # Import tqdm for progress bar

base_path = "/home/sebastian/Documents/Masterarbeit/Daten/"

# Domestic Documents
for year in range(1990, 2024):
    metadata_file_path = f"{base_path}{year}VAERSDATA.csv"
    vax_file_path = f"{base_path}{year}VAERSVAX.csv"
    symptoms_file_path = f"{base_path}{year}VAERSSYMPTOMS.csv"
    
    metadata_df = pd.read_csv(metadata_file_path, encoding='ISO-8859-1', low_memory=False)
    vax_df = pd.read_csv(vax_file_path, encoding='ISO-8859-1', low_memory=False, usecols=['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'VAX_ROUTE', 'VAX_SITE' ,'VAX_NAME'])
    symptoms_df = pd.read_csv(symptoms_file_path, encoding='ISO-8859-1', low_memory=False, usecols=['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5'])
    
    metadata_df = metadata_df.drop(['V_FUNDBY', 'FORM_VERS'], axis=1)
    
    
    metadata_df['AGE'] = metadata_df.apply(calculate_age, axis=1) # Insert the 'AGE' column as the 4th column
    metadata_df.insert(3, 'AGE', metadata_df.pop('AGE'))
    metadata_df['TODAYS_DATE'] = metadata_df.apply(form_completed, axis=1)
    metadata_df[['RECVDATE', 'DATEDIED', 'VAX_DATE', 'ONSET_DATE', 'TODAYS_DATE']] = metadata_df[['RECVDATE', 'DATEDIED', 'VAX_DATE', 'ONSET_DATE', 'TODAYS_DATE']].apply(lambda x: x.map(convert_date))
    metadata_df = metadata_df.drop(['AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'RPT_DATE'], axis=1)
    metadata_df.insert(2, 'TODAYS_DATE', metadata_df.pop('TODAYS_DATE'))
    metadata_df['ALLERGIES'] = metadata_df['ALLERGIES'].astype(str)
    metadata_df['ALLERGIES'] = metadata_df['ALLERGIES'].apply(lambda x: correct_spelling(x) if x != 'nan' else np.nan)
    metadata_df['SERIOUS'] = np.where(metadata_df[['DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT']].eq('Y').any(axis=1), 'Y', 'N')
    metadata_df.insert(6, 'SERIOUS', metadata_df.pop('SERIOUS'))

    columns = ['SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']
    for col in columns:
        symptoms_df[col] = replace_vaccination_site(symptoms_df[col])
    
    for index, row in tqdm(metadata_df.iterrows(), total=len(metadata_df), desc=f"Processing Year {year}"):  # Use tqdm for progress bar
        entry = {}
        vax_entries = [] 
        symptoms_entry = []
    
        # Add metadata columns directly to the entry
        for column in metadata_df.columns:
            if column == 'OTHER_MEDS':
                # Split the string on the comma or semicolon and store as a list
                if pd.notna(row[column]):
                    meds = re.split(',|;', row[column])  # Use regex to split on both delimiters
                    entry['OTHER_MEDS'] = [med.strip() for med in meds if med.strip()]  # List of non-empty stripped meds
                else:
                    entry['OTHER_MEDS'] = np.nan
            elif column == 'CUR_ILL':
                # Split the string on the comma and create a dictionary
                if pd.notna(row['CUR_ILL']):
                    ills = re.split(',|;', row['CUR_ILL'])
                    entry['CUR_ILL'] = [ill.strip() for ill in ills if ill.strip()]
                else:
                     entry['CUR_ILL'] = np.nan   
            elif column == 'HISTORY':
                # Split the string on the comma and create a dictionary
                if pd.notna(row[column]):
                    history = re.split(',|;', row[column])
                    entry['HISTORY'] = [his.strip() for his in history if his.strip()]
                else:
                    entry['HISTORY'] = np.nan
                    
            elif pd.notna(row[column]):
                entry[column] = row[column]
            else:
                entry[column] = np.nan
        
        # Filter vax_df based on VAERS_ID
        vax_data = vax_df[vax_df['VAERS_ID'] == row['VAERS_ID']] 
        vax_data = vax_data.drop(['VAERS_ID'], axis=1)
        
        # Check if vax_data is not empty
        if not vax_data.empty:
            # Iterate over each vaccine entry and add it to the list
            for vax_row in vax_data.itertuples(index=False):
                vax_entry_i = {column: getattr(vax_row, column) if pd.notna(getattr(vax_row, column)) else np.nan for column in vax_data.columns}
                if vax_entry_i["VAX_MANU"] != "UNKNOWN MANUFACTURER":
                    vax_entries.append(vax_entry_i)
        if len(vax_entries) == 0:
            continue
            
        entry['vax_data'] = vax_entries  # Add the list of vaccine entries to the main entry
                
         # Filter symptoms_df based on VAERS_ID
        symptoms_data = symptoms_df[symptoms_df['VAERS_ID'] == row['VAERS_ID']] 
        symptoms_data = symptoms_data.drop(['VAERS_ID'], axis=1)
        
        # Check if symptoms_data is not empty
        if not symptoms_data.empty:
            # Construct a list of symptoms, excluding any NaN values
            symptoms_list = [value for value in symptoms_data.values.flatten() if pd.notna(value)]
            entry['symptoms'] = symptoms_list
        else:
            entry['symptoms'] = np.nan
            
                
        # Upload entry to MongoDB
        collection.insert_one(entry)
        

Processing Year 2020: 100%|██████████| 49644/49644 [03:49<00:00, 216.67it/s]


In [ ]:
entry  

# non-domestic reports

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

# Set base path
base_path = "/home/sebastian/Documents/Masterarbeit/Daten/"

# File paths
metadata_file_path = f"{base_path}NonDomesticVAERSDATA.csv"
vax_file_path = f"{base_path}NonDomesticVAERSVAX.csv"
symptoms_file_path = f"{base_path}NonDomesticVAERSSYMPTOMS.csv"

metadata_df = pd.read_csv(metadata_file_path, encoding='ISO-8859-1', low_memory=False)
vax_df = pd.read_csv(vax_file_path, encoding='ISO-8859-1', low_memory=False, usecols=['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'VAX_ROUTE', 'VAX_SITE' ,'VAX_NAME'])
symptoms_df = pd.read_csv(symptoms_file_path, encoding='ISO-8859-1', low_memory=False, usecols=['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5'])

metadata_df = metadata_df.drop(['V_FUNDBY', 'FORM_VERS'], axis=1)


metadata_df['AGE'] = metadata_df.apply(calculate_age, axis=1) # Insert the 'AGE' column as the 4th column
metadata_df.insert(3, 'AGE', metadata_df.pop('AGE'))
metadata_df['TODAYS_DATE'] = metadata_df.apply(form_completed, axis=1)
metadata_df[['RECVDATE', 'DATEDIED', 'VAX_DATE', 'ONSET_DATE', 'TODAYS_DATE']] = metadata_df[['RECVDATE', 'DATEDIED', 'VAX_DATE', 'ONSET_DATE', 'TODAYS_DATE']].apply(lambda x: x.map(convert_date))
metadata_df = metadata_df.drop(['AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'RPT_DATE'], axis=1)
metadata_df.insert(2, 'TODAYS_DATE', metadata_df.pop('TODAYS_DATE'))
metadata_df['ALLERGIES'] = metadata_df['ALLERGIES'].astype(str)
metadata_df['ALLERGIES'] = metadata_df['ALLERGIES'].apply(lambda x: correct_spelling(x) if x != 'nan' else np.nan)
metadata_df['SERIOUS'] = np.where(metadata_df[['DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT']].eq('Y').any(axis=1), 'Y', 'N')
metadata_df.insert(6, 'SERIOUS', metadata_df.pop('SERIOUS'))

columns = ['SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']
for col in columns:
    symptoms_df[col] = replace_vaccination_site(symptoms_df[col])
    
# Create a progress bar
progress_bar = tqdm(total=len(metadata_df), desc="Processing rows")

# Iterate through rows of metadata_df
for index, row in metadata_df.iterrows():
        entry = {}
        vax_entries = [] 
        symptoms_entry = []
    
        # Add metadata columns directly to the entry
        for column in metadata_df.columns:
            if column == 'OTHER_MEDS':
                # Split the string on the comma or semicolon and store as a list
                if pd.notna(row[column]):
                    meds = re.split(',|;', row[column])  # Use regex to split on both delimiters
                    entry['OTHER_MEDS'] = [med.strip() for med in meds if med.strip()]  # List of non-empty stripped meds
                else:
                    entry['OTHER_MEDS'] = np.nan
            elif column == 'CUR_ILL':
                # Split the string on the comma and create a dictionary
                if pd.notna(row['CUR_ILL']):
                    ills = re.split(',|;', row['CUR_ILL'])
                    entry['CUR_ILL'] = [ill.strip() for ill in ills if ill.strip()]
                else:
                     entry['CUR_ILL'] = np.nan   
            elif column == 'HISTORY':
                # Split the string on the comma and create a dictionary
                if pd.notna(row[column]):
                    history = re.split(',|;', row[column])
                    entry['HISTORY'] = [his.strip() for his in history if his.strip()]
                else:
                    entry['HISTORY'] = np.nan
                    
            elif pd.notna(row[column]):
                entry[column] = row[column]
            else:
                entry[column] = np.nan
        
        # Filter vax_df based on VAERS_ID
        vax_data = vax_df[vax_df['VAERS_ID'] == row['VAERS_ID']] 
        vax_data = vax_data.drop(['VAERS_ID'], axis=1)
        
        # Check if vax_data is not empty
        if not vax_data.empty:
            # Iterate over each vaccine entry and add it to the list
            for vax_row in vax_data.itertuples(index=False):
                vax_entry_i = {column: getattr(vax_row, column) if pd.notna(getattr(vax_row, column)) else np.nan for column in vax_data.columns}
            if vax_entry_i["VAX_MANU"] != "UNKNOWN MANUFACTURER":
                    vax_entries.append(vax_entry_i)
        if len(vax_entries) == 0:
            continue
        
        entry['vax_data'] = vax_entries  # Add the list of vaccine entries to the main entry
                
         # Filter symptoms_df based on VAERS_ID
        symptoms_data = symptoms_df[symptoms_df['VAERS_ID'] == row['VAERS_ID']] 
        symptoms_data = symptoms_data.drop(['VAERS_ID'], axis=1)
        
        # Check if symptoms_data is not empty
        if not symptoms_data.empty:
            # Construct a list of symptoms, excluding any NaN values
            symptoms_list = [value for value in symptoms_data.values.flatten() if pd.notna(value)]
            entry['symptoms'] = symptoms_list
        else:
            entry['symptoms'] = np.nan
                
        # Upload entry to MongoDB
        collection.insert_one(entry)

        # Update the progress bar
        progress_bar.update(1)

# Close the progress bar once finished
progress_bar.close()


In [21]:
entry

{'VAERS_ID': 25002,
 'RECVDATE': Timestamp('1990-07-02 00:00:00'),
 'TODAYS_DATE': nan,
 'STATE': 'FR',
 'AGE': 82.0,
 'SEX': 'M',
 'SERIOUS': 'Y',
 'DIED': nan,
 'DATEDIED': nan,
 'L_THREAT': nan,
 'ER_VISIT': nan,
 'HOSPITAL': nan,
 'HOSPDAYS': nan,
 'X_STAY': nan,
 'DISABLE': 'Y',
 'RECOVD': nan,
 'VAX_DATE': Timestamp('1989-11-20 00:00:00'),
 'ONSET_DATE': Timestamp('1989-11-20 00:00:00'),
 'NUMDAYS': 0.0,
 'OTHER_MEDS': ['Thioridazine', 'Triazolam'],
 'CUR_ILL': nan,
 'HISTORY': ['Senile dementia', 'Diabetes mellitus', 'seizures'],
 'PRIOR_VAX': '~ ()~~~In patient',
 'BIRTH_DEFECT': nan,
 'OFC_VISIT': nan,
 'ER_ED_VISIT': nan,
 'ALLERGIES': nan,
 'vax_data': [{'VAX_TYPE': 'PPV',
   'VAX_MANU': 'MERCK & CO. INC.',
   'VAX_LOT': 'M0870',
   'VAX_DOSE_SERIES': 'UNK',
   'VAX_NAME': 'PNEUMO (PNEUMOVAX)'}],
 'symptoms': ['Chills', 'Convulsion', '', '', '']}